In [1]:
print('ok')

ok


In [1]:
import os 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ['GROQ_API_KEY']

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [3]:
loader = TextLoader("new.txt")
documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [5]:
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

pip install sentence-transformers

In [6]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13040\3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\ProgramData\anaconda3\envs\prac\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
retriever = FAISS.from_documents(texts, embeddings).as_retriever(search_kwargs={"k": 20})


In [9]:
query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.invoke(query)
print(docs)

[Document(id='172bc3ca-6daa-4e57-9c8e-449d6a05f861', metadata={'source': 'new.txt', 'id': 73}, page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'), Document(id='ba6d59c3-da0a-440d-a31b-c43289725cf0', metadata={'source': 'new.txt', 'id': 72}, page_content='We cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Jus

Doing reranking with FlashRank

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_groq import ChatGroq
from flashrank import Ranker


In [11]:
llm = ChatGroq(model = "llama3-70b-8192")

In [12]:
# Ensure FlashrankRerank is properly built before using it
FlashrankRerank.model_rebuild()

# Now create an instance
compressor = FlashrankRerank()

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [01:51<00:00, 930kiB/s] 


In [14]:
compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
print([doc.metadata["id"] for doc in compressed_docs])



[73, 2, 35]


In [19]:
print(compressed_docs[1])

page_content='He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. 

In this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight.' metadata={'id': 2, 'relevance_score': 0.98592573, 'source': 'new.txt'}
